In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import  StandardScaler 
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report



### Read Data

In [2]:

diabetes = pd.read_csv("diabetes_binary_health_indicators_BRFSS2015.csv")
diabetes.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,1.0,1.0,40.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,5.0,18.0,15.0,1.0,0.0,9.0,4.0,3.0
1,0.0,0.0,0.0,0.0,25.0,1.0,0.0,0.0,1.0,0.0,...,0.0,1.0,3.0,0.0,0.0,0.0,0.0,7.0,6.0,1.0
2,0.0,1.0,1.0,1.0,28.0,0.0,0.0,0.0,0.0,1.0,...,1.0,1.0,5.0,30.0,30.0,1.0,0.0,9.0,4.0,8.0
3,0.0,1.0,0.0,1.0,27.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,11.0,3.0,6.0
4,0.0,1.0,1.0,1.0,24.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,3.0,0.0,0.0,0.0,11.0,5.0,4.0


### Remove Outliers

In [3]:
def drop_outliers_iqr(df, column):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    df_cleaned = df[(df[column] >= lower_bound) & (df[column] <= upper_bound)]
    return df_cleaned

diabetes_cleaned = drop_outliers_iqr(diabetes, 'BMI')
diabetes_cleaned = drop_outliers_iqr(diabetes_cleaned, 'GenHlth')

### Split Data

In [4]:

x_train, x_test = train_test_split(diabetes_cleaned, test_size=0.20, random_state=42)
x_test,y_test = x_test.drop(['Diabetes_binary'],axis=1),x_test['Diabetes_binary']
x_train,y_train = x_train.drop(['Diabetes_binary'],axis=1),x_train['Diabetes_binary']


### Standardize Data

In [5]:
# scaler = StandardScaler()
# scaler = RobustScaler()
scaler = StandardScaler()

scaler.fit(x_train)
x_train_scaler = scaler.transform(x_train)
x_test_scaler = scaler.transform(x_test)

### Adaboost model

In [15]:

selector = SelectKBest(f_classif, k=10)
X_new = selector.fit_transform(x_train, y_train)

selected_features = x_train.columns[selector.get_support()]
print("Best features:", selected_features)

Best features: Index(['HighBP', 'HighChol', 'BMI', 'HeartDiseaseorAttack', 'GenHlth',
       'PhysHlth', 'DiffWalk', 'Age', 'Education', 'Income'],
      dtype='object')


In [17]:
X_train_selected = selector.transform(x_train)
X_test_selected = selector.transform(x_test)

scaler = StandardScaler()

scaler.fit(X_train_selected)
x_train_scaler = scaler.transform(X_train_selected)
x_test_scaler = scaler.transform(X_test_selected)


model_selected_features = AdaBoostClassifier(
    estimator=LogisticRegression(class_weight="balanced"),
    n_estimators=100,
    random_state=42
)

model_selected_features.fit(x_train_scaler, y_train)

y_train_pred = model_selected_features.predict(x_train_scaler)
y_test_pred = model_selected_features.predict(x_test_scaler)


train_accuracy = accuracy_score(y_train, y_train_pred) * 100
test_accuracy = accuracy_score(y_test, y_test_pred) * 100

print(f"Training Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")
print(classification_report(y_test, y_test_pred, target_names=['No Diabetes', 'Diabetes']))

Training Accuracy: 72.76%
Test Accuracy: 72.76%
              precision    recall  f1-score   support

 No Diabetes       0.96      0.72      0.82     40903
    Diabetes       0.28      0.76      0.41      5689

    accuracy                           0.73     46592
   macro avg       0.62      0.74      0.61     46592
weighted avg       0.87      0.73      0.77     46592



## Bias and Variance

In [13]:
from bias_variance import get_bias_variance
mse, bias, var = get_bias_variance(
    model_selected_features, x_train_scaler, y_train,x_test_scaler,y_test)



In [14]:
print("mse is :",mse)
print("biase is :",bias)
print("variance is :",var)

mse is : 0.2723944024725275
biase is : 0.2693444368131868
variance is : 0.003049965659340656
